In [ ]:
import cv2
import os
from operator import itemgetter
from numpy import array
import csv
import time
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from sklearn.metrics import confusion_matrix

In [ ]:
def import_dataset(path_dataset, mode, val=False):
    dataset = []
    dataset_unsorted = []
    print("Start importing " + mode + " images...")
    for filename in os.listdir(path_dataset):
        if filename.endswith(".jpg"):
            complete_path = os.path.join(path_dataset, filename)
            image = cv2.imread(complete_path, cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # from BGR to RGB
            dim = (IMG_HEIGHT, IMG_WIDTH)  # image dimensions
            image = cv2.resize(image, dsize=dim, interpolation=cv2.INTER_AREA)
            image_filename = [filename, image]
            dataset_unsorted.append(image_filename)
        else:
            continue

    # orders the list of images by alphabetic order of its name
    dataset_unsorted = sorted(dataset_unsorted, key=itemgetter(0))

    # creates a list with only the image and not its filename
    for x in dataset_unsorted:
        dataset.append(x[1])
    return array(dataset)  # Converts the lists into numpy.ndarray

In [ ]:
def assign_labels(path_groundtruth):
    target = []
    counter = {'BKL': 0, 'DF': 0, 'VASC': 0}
    i = 0
    with open(path_groundtruth, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            if i == 0:
                i += 1
                continue

            if row[1] == '1.0': # BKL
                counter['BKL'] += 1
                target.append(0)
            elif row[2] == '1.0': # DF
                counter['DF'] += 1
                target.append(1)
            elif row[3] == '1.0':  # VASC
                counter['VASC'] += 1
                target.append(2)
    print(counter)
    file.close()
    return counter, target

In [ ]:
def plot_val_train_error(fit):
    plt.plot(fit.history['accuracy'])
    plt.plot(fit.history['val_accuracy'])
    plt.grid(True)
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

    plt.plot(fit.history['loss'])
    plt.plot(fit.history['val_loss'])
    plt.grid(True)
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [ ]:
def create_model_d(model = 'densenet', transferLearning=True):
    if model == 'resnet':

        from tensorflow.keras.applications.resnet import ResNet101
        from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

        if transferLearning == False:
            resnet = ResNet101(include_top=False, weights=None, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
        else:
            resnet = ResNet101(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

        model = tf.keras.Sequential(resnet)
        model.add(GlobalAveragePooling2D())
        model.add(tf.keras.layers.Dropout(0.5))
        model.add(Dense(units=3, activation="softmax"))

    elif model == 'densenet':

        from tensorflow.keras.applications.densenet import DenseNet121
        from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

        if transferLearning == False:
            densenet = DenseNet121(include_top=False, weights=None, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
        else:
            densenet = DenseNet121(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

        model = tf.keras.Sequential(densenet)
        model.add(GlobalAveragePooling2D())
        model.add(tf.keras.layers.Dropout(0.5))
        model.add(Dense(units=3, activation="softmax"))

    else:
        print("That model is not available.")
        exit(0)

    return model

In [ ]:
# Learning Rate Scheduler
def scheduler(epoch, noEpochs, learningRate):
    if epoch < int(0.5 * noEpochs):
        return learningRate
    elif epoch < int(0.75 * noEpochs):
        return learningRate / 10
    else:
        return learningRate / 100

In [ ]:
# Import Training and Validation datasets and Groundtruth of classifier d
pathTrain = '/home/ruben/Desktop/smalltrain2018'
truthTrain = '/home/ruben/Desktop/HierSmall/d/labels.csv'

x_train = import_dataset(pathTrain, 'training')
counter, y_train = assign_labels(truthTrain)

pathVal  = '/home/ruben/Desktop/smallval2018'
truthVal  = '/home/ruben/Desktop/HierSmall/val/d/labels.csv'

x_val = import_dataset(pathVal, 'validation')
counter, y_val = assign_labels(truthVal)

print("Dataset Imported")

In [ ]:
# Define training variables
noEpochs = 20
learningRate = 1e-5
noClasses = 2
batchSize = 10
modelName = 'densenet'

checkpointPath = {
    'resnet': "/home/ruben/Desktop/Small/Hier/d/resnet/cp.ckpt",
    'densenet': "/home/ruben/Desktop/Small/Hier/d/densenet/cp.ckpt"
}

w_0_float = float(counter['BKL'])
w_1_float = float(counter['DF'])
w_2_float = float(counter['VASC'])
w_total = w_0_float + w_1_float + w_2_float

w_0 = w_total / w_0_float
w_1 = w_total / w_1_float
w_2 = w_total / w_2_float
classWeights = {0: w_0, 1: w_1, 2: w_2}

y_train_cat = keras.utils.to_categorical(y_train, noClasses)
y_val_cat = keras.utils.to_categorical(y_val, noClasses)

# Callback to adapt the learning rate value
schedulerLearningRate = tf.keras.callback.LearningRateScheduler(scheduler)

# Callback to save the Best Model Weights
checkpoint_dir = os.path.dirname(checkpointPath[modelName])
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpointPath[modelName],
    save_weights_only=True,
    verbose=1, 
    save_best_only=True
)

In [ ]:
# Create and fit the model
model = create_model(modelName, transferLearning=True)
model.summary()

model.compile(
    loss = keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(lr=1e-5),
    metrics = ['accuracy']
)

model.fit(
    x_train,
    y_train_cat,
    batch_size=batchSize,
    class_weight=classWeights,
    callbacks=[schedulerLearningRate, checkpoint],
    epochs = noEpochs,
    shuffle=True
    validation_data=(x_val, y_val_cat)
)

In [ ]:
# Inference Stage
model.load_weights(checkpointPath[modelName])

y_pred = model.predict_classes(x_val)
conf_matrix = confusion_matrix(y_val, y_pred)
print(conf_matrix)

score = model.evaluate(x_val, y_val_cat, verbose=1)
print(f'Val loss: {score[0]} / Val accuracy: {score[1]}')

plot_val_train_error(fit)